# Top Restaurantes do bairro Pinheiros/SP via Zomato

### Importação de pacotes:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
response = requests.get("https://www.zomato.com/pt/sao-paulo-sp/best-pinheiros-restaurants", headers=headers)
html = response.content

In [ ]:
type(html)

In [ ]:
html = html.decode('utf-8')

In [ ]:
type(html)

In [ ]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

html = trata_html(html)
html

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
soup

In [ ]:
type(soup)

## Entendendo o site:

In [ ]:
print(soup.prettify())

In [ ]:
restaurantes = soup.find('div', {'class': 'ui cards'})
restaurantes

In [ ]:
dados_restaurante = restaurantes.find('div', {'class':'col-s-16 col-m-12 pl0'})
dados_restaurante

In [ ]:
tipo_restaurante = restaurantes.find('div', {'class':'search-page-text clearfix row'})
tipo_restaurante

In [ ]:
card = []
card = {}

In [ ]:
card['restaurante'] = [dados_restaurante.find('a', {'class':'result-title hover_feedback zred bold ln24 fontsize0'}).getText()]
card['nota'] = [dados_restaurante.find('span', {'class':'rating-value'}).getText()]
card['numero_de_avaliacoes'] = [dados_restaurante.find('span', {'class':'review-count medium'}).getText()]
card['tipo'] = [dados_restaurante.find('a', {'class':'zdark ttupper fontsize6'}).getText()]
card['endereco'] = [dados_restaurante.find('div', {'class':'col-m-16 search-result-address grey-text nowrap ln22'}).getText()]
card['cozinha'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}).getText()]
card['custo_para_dois'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 pl0'}).getText()]

In [ ]:
card

In [ ]:
restaurantes_pinheiros = pd.DataFrame(card, index=None)
restaurantes_pinheiros

## Coletando os dados e transformando em um DataFrame:

In [ ]:
# Declarando variável cards
cards = []

# Obtendo o HTML:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
response = requests.get("https://www.zomato.com/pt/sao-paulo-sp/best-pinheiros-restaurants", headers=headers)
html = response.content

# Decodificando o HTML:
html = html.decode('utf-8')

# Tratando o HTML:
html = trata_html(html)

#
soup = BeautifulSoup(html, 'html.parser')


# Obtendo as informações de cada um dos restaurantes:
dados_restaurantes = soup.find('div', {'class': 'ui cards'}).findAll('div', class_='col-s-16 col-m-12 pl0')
tipos_restaurantes = soup.find('div', {'class': 'ui cards'}).findAll('div', class_='search-page-text clearfix row')


# Coletando as informações de cada restaurante:
n_restaurantes_pagina = len(dados_restaurantes)

for i in range(n_restaurantes_pagina):
    card = {}
    
    # Informações sobre os restaurantes:
    card['restaurante'] = dados_restaurantes[i].find('a', {'class':'result-title hover_feedback zred bold ln24 fontsize0'}).getText()
    if dados_restaurantes[i].find('span', {'class':'rating-value'}) is not None:
        card['nota'] = dados_restaurantes[i].find('span', {'class':'rating-value'}).getText()
    else:
        'None'
    if dados_restaurantes[i].find('span', {'class':'review-count medium'}) is not None:
        card['numero_de_avaliacoes'] = dados_restaurantes[i].find('span', {'class':'review-count medium'}).getText()
    else:
        'None'
    if dados_restaurantes[i].find('a', {'class':'zdark ttupper fontsize6'}) is not None:
        card['tipo'] = dados_restaurantes[i].find('a', {'class':'zdark ttupper fontsize6'}).getText()
    else:
        'None'
    if tipos_restaurantes[i].find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}) is not None:
        card['cozinha'] = tipos_restaurantes[i].find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}).getText()
    else:
        'None'
    if tipos_restaurantes[i].find('span', {'class':'col-s-11 col-m-12 pl0'}) is not None:
        card['custo_para_dois'] = tipos_restaurantes[i].find('span', {'class':'col-s-11 col-m-12 pl0'}).getText()
    else:
        'None'
    
    # Adicionando resultado a lista CARD
    cards.append(card)
    
# Criando um DataFrame com os resultados
dataset_restaurantes = pd.DataFrame(cards)
dataset_restaurantes.to_csv('dataset_restaurantes.csv', sep=';', index=False, encoding = 'utf-8-sig')
dataset_restaurantes